# Headline Analysis

We have 2 news headline datasets,
1. data/News_Category_Dataset_v3.json 
    - HuffPost
    - 210,294 records between 2012 and 2022
    - Attributes: (category, headline, authors, link, short_description, date)
2. data/abcnews-date-text.csv 
    - Australian ABC
    - 1,103,665 records between 2003 and 2021
    - Attributes: (headline, date)